<a href="https://colab.research.google.com/github/hanlululu/02807-Computational-tools-for-Data-Science/blob/main/Week8/exercises_week_8_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02807 - Week 8 Exercises:  More Spark DataFrame API

In this exercise session you will continue learning PySpark running it in local mode.

These exercises are inspired by [Tiziano Piccardi](http://piccardi.me/) and [this notebook](https://github.com/epfl-ada/2019/blob/master/Tutorials/04%20-%20Scaling%20Up/PySpark.ipynb) (which contains partial solutions). 

# Setup

Setup spark for running in local mode using these instructions.

In [ ]:
!pip install pyspark

# Instructions on p. 20 Learning Spark, 2nd ed.
# Here's a quick-guide, googling may also be required
# 1) Install pyspark via conda/pip
#          pyspark requires the JAVA_HOME environment variable is set.
# 2) Install JDK 8 or 11, figure out the install location
#          Suggest to use https://adoptopenjdk.net/
# 3) Update the JAVA_HOME environment variable set programmatically below 
#    with your install location specifics

# JAVA_HOME environment variable is set programatically below
# but you must point it to your local install

import os
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home/"

# If you get "Job aborted due to stage failure" and 
# "Python worker failed to connect back." exceptions, 
# this should be solved by additionally setting these 
# environment variables

# os.environ['PYSPARK_PYTHON'] = 'python'
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
# os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'notebook'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext, SparkConf

In [ ]:
# create the Spark session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

spark

# Vietnam War Dataset

**Pres. Johnson**: _What do you think about this Vietnam thing? I’d like to hear you talk a little bit._

**Sen. Russell**: _Well, frankly, Mr. President, it’s the damn worse mess that I ever saw, and I don’t like to brag and I never have been right many times in my life, but I knew that we were going to get into this sort of mess when we went in there._

May 27, 1964

![banner](https://raw.githubusercontent.com/epfl-ada/2019/c17af0d3c73f11cb083717b7408fedd86245dc4d/Tutorials/04%20-%20Scaling%20Up/img/banner.jpg)

----

The Vietnam War, also known as the Second Indochina War, and in Vietnam as the Resistance War Against America or simply the American War, was a conflict that occurred in Vietnam, Laos, and Cambodia from 1 November 1955 to the fall of Saigon on 30 April 1975. It was the second of the Indochina Wars and was officially fought between North Vietnam and the government of South Vietnam.

**The dataset describes all the air force operation in during the Vietnam War.**

**Bombing_Operations** [Get the dataset here](https://courses.compute.dtu.dk/02807/2021/lectures/week8/Bombing_Operations.json.gz)

- AirCraft: _Aircraft model (example: EC-47)_
- ContryFlyingMission: _Country_
- MissionDate: _Date of the mission_
- OperationSupported: _Supported War operation_ (example: [Operation Rolling Thunder](https://en.wikipedia.org/wiki/Operation_Rolling_Thunder))
- PeriodOfDay: _Day or night_
- TakeoffLocation: _Take off airport_
- TimeOnTarget
- WeaponType
- WeaponsLoadedWeight

**Aircraft_Glossary** [Get the dataset here](https://courses.compute.dtu.dk/02807/2021/lectures/week8/Aircraft_Glossary.json.gz)

- AirCraft: _Aircraft model (example: EC-47)_
- AirCraftName
- AirCraftType

**Dataset Information:**

THOR is a painstakingly cultivated database of historic aerial bombings from World War I through Vietnam. THOR has already proven useful in finding unexploded ordnance in Southeast Asia and improving Air Force combat tactics:
https://www.kaggle.com/usaf/vietnam-war-bombing-operations

Load the datasets (this may require a bit of patience).

In [ ]:
df_aircraft = spark.read.json('Aircraft_Glossary.json.gz')

In [ ]:
df_operations = spark.read.json('Bombing_Operations.json.gz')

Display the schemas and inspect some rows of `df_operations`.

In [ ]:
# your code goes here
df_aircraft.printSchema()

In [ ]:
# your code goes here
df_operations.printSchema()

In [ ]:
# your code goes here
df_operations.limit(6).toPandas()

How many bombing operations are described in `df_operations`?

In [ ]:
# your code goes here
f"A total of {df_operations.count()} operations are described"

# Question 1: Which countries are involved and in how many missions? 

Keywords: `Dataframe API`, `SQL`, `group by`, `sort`

The country associated with the operation is found in the `ContryFlyingMission` [sic] field. Display the result using `show`.

In [ ]:
# your code goes here
df_operations.groupBy('ContryFlyingMission').count().show()

Plot a horizontal bar chart with the number of operations by country. Consider using a logarithmic scale due to the skewedness of per country operations.

In [ ]:
# your code goes here
df_operations_count = df_operations \
                        .groupBy('ContryFlyingMission') \
                        .count() \
                        .sort(F.asc('count')) \
                        .toPandas()

_ = df_operations_count.set_index('ContryFlyingMission').plot.barh(figsize=(16, 4), log=True)

# Question 2: Show the number of missions in time for each of the countries involved.

Transform the dataframe to contain only relevant columns for this query. Observe that `MissionDate` field is of type string. You can use `F.to_date` to convert it to a `DateType`.

In [ ]:
# your code goes here
df_operations_q2 = df_operations.select(F.col('ContryFlyingMission'), 
                                        F.to_date(F.col('MissionDate'), 'yyyy-MM-dd').alias('MissionDate'))

Now compute the count of missions per country for each `MissionDate` and sort the dataframe by date (ascending).

In [ ]:
# your code goes here
df_operations_q2_counts = df_operations_q2 \
                            .groupBy(['ContryFlyingMission', 'MissionDate']) \
                            .count() \
                            .sort(F.asc('MissionDate'))

Now we can plot the content with a different series for each country (using an appropriate plot type). To this end, you may use the pattern from "Slides Week 4" where we did a scatter plot colored by country.

In [ ]:
# your code goes here
df_operations_q2_counts_pd = df_operations_q2_counts.toPandas()

fig, ax = plt.subplots(figsize=(16,4))
for label, df in df_operations_q2_counts_pd.groupby(by='ContryFlyingMission'):
    ax.plot(df['MissionDate'], df['count'], label=label)

_ = ax.legend()

We can observe how South Vietnam increased its missions starting from 1970. The drop in 1973 is motivated by the [Paris Peace Accords](https://en.wikipedia.org/wiki/Paris_Peace_Accords) that took place on January 27th, 1973, to establish peace in Vietnam and end the war.

The plot looks a bit erratic, as the count is computed for each date. We can alleviate this by instead having the x-axis be months, and the y-axis be the count of operations in that month. Recompute such a dataframe and plot it once more. To this end, `F.trunc` will come in handy.

In [ ]:
# your code goes here
df_operations_q2_counts_months_pd = \
    df_operations_q2 \
        .withColumn('MissionYearMonth', F.trunc('MissionDate', 'mm')) \
        .groupBy(['ContryFlyingMission', 'MissionYearMonth']) \
        .count() \
        .sort(F.asc('MissionYearMonth')) \
        .toPandas()

fig, ax = plt.subplots(figsize=(16,4))
for label, df in df_operations_q2_counts_months_pd.groupby(by='ContryFlyingMission'):
    ax.plot(df['MissionYearMonth'], df['count'], label=label)

_ = ax.legend()

----

# Question 3: Who bombed this location?

<img style="float: right;" src="https://raw.githubusercontent.com/epfl-ada/2019/c17af0d3c73f11cb083717b7408fedd86245dc4d/Tutorials/04%20-%20Scaling%20Up/img/Hanoi_POL1966.jpg">

This picture is the Hanoi POL facility (North Vietnam) burning after it was attacked by the U.S. Air Force on 29 June 1966 in the context of the Rolling Thunder operation. 

We are interested in discovering what was the most common take-off location during that day. Transform the operations dataframe to contain operations for the date in question, then cache and execute this caching.

In [ ]:
# your code goes here
cond = (F.col('MissionDate') == '1966-06-29') & (F.col('TargetCountry') == 'NORTH VIETNAM')
df_operations_jun_29 = df_operations.filter(cond)

Which coutries scheduled operations that day?

In [ ]:
# your code goes here
df_operations_jun_29.select(F.col('ContryFlyingMission')).distinct().show(truncate=False)

How many operations where done per country on this date?

In [ ]:
# your code goes here
df_operations_jun_29.groupby('ContryFlyingMission').count().show(truncate=False)

Time the previous query using the `collect` action. Then cache the dataframe containing operations from the date in question, invoke an action and time the previous query on this cached dataframe. How much faster is the query on your cached dataframe?

In [ ]:
# your code goes here
%timeit df_operations_jun_29.groupby('ContryFlyingMission').count().collect()
df_operations_jun_29.cache()
df_operations_jun_29.count()
%timeit df_operations_jun_29.groupby('ContryFlyingMission').count().collect()

Write the cached dataframe to disk and read it back (you needn't assign it to a variable when reading it in).

In [ ]:
# your code goes here
df_operations_jun_29.write.json('operations_jun_29.json')
spark.read.json('operations_jun_29.json')

Show the number of operations per takeoff location, in descending order.

In [ ]:
# your code goes here
df_operations_jun_29.groupby('TakeoffLocation').count().sort(F.desc('count')).show()

<img style="float: right;" src="https://raw.githubusercontent.com/epfl-ada/2019/c17af0d3c73f11cb083717b7408fedd86245dc4d/Tutorials/04%20-%20Scaling%20Up/img/USS_Constellation.jpg">


That day the most common take-off location was the ship USS Constellation (CV-64). We cannot univocally identify one take off location, but we can reduce the possible candidates. Next steps: explore TimeOnTarget feature.

_USS Constellation (CV-64), a Kitty Hawk-class supercarrier, was the third ship of the United States Navy to be named in honor of the "new constellation of stars" on the flag of the United States. One of the fastest ships in the Navy, as proven by her victory during a battlegroup race held in 1985, she was nicknamed "Connie" by her crew and officially as "America's Flagship"._

Finally, compute the mean, median first and third quantile of the `TimeOnTarget` value per takeoff location (you could make use of `F.percentile_approx` from pyspark 3.1.1 and up).  

In [ ]:
# your code goes here
df_operations_jun_29.groupby('TakeoffLocation') \
    .agg(F.mean('TimeOnTarget').alias('avg_tot'), 
         F.percentile_approx('TimeOnTarget', [0.25, 0.5, 0.75], 1000000).alias('quantiles')) \
    .sort(F.asc('avg_tot')) \
    .toPandas()

----

# Question 4: What is the most used aircraft type during the Vietnam war (number of operations)?


Let's check the content of `Aircraft_Glossary`. Have a look at it using `show`.

In [ ]:
# your code goes here
df_aircraft.show()

How many unique values do we have in the `AirCraftType` column? Display them.

In [ ]:
# your code goes here
df_aircraft.select('AirCraftType').distinct().toPandas()

Both our dataframes has a column called `AirCraft` (a shared key between the two). Join the dataframes together and name the result `df_operations_aircrafts`.

In [ ]:
# your code goes here
df_operations_aircrafts = df_operations.join(df_aircraft, on='AirCraft', how='inner')
df_operations_aircrafts.limit(10).toPandas()

Determine the most used aircraft types for the entirety of the Vietnam War.

In [ ]:
# your code goes here
df_operations_aircrafts.groupby('AirCraftType').count().sort(F.desc('count')).show(truncate=False)

Note: This dataset would require further cleaning and normalization. See `Fighter Jet Bomber`, `Jet Fighter Bomber`, `Fighter bomber jet`

Determine which aircraft types were fastest (on average) to complete their operations.

In [ ]:
# your code goes here
df_operations_aircrafts.groupby('AirCraftType') \
    .agg(F.avg('TimeOnTarget').alias('avg_tot')) \
    .sort(F.asc('avg_tot')) \
    .toPandas()

What join strategy was chosen by spark for this last query? (Inspect the Spark UI SQL query DAG). 

Finally, use the `hint` function to request spark to do a Shuffle Merge Sort Join (SMJ) for the previous query (see [here](https://towardsdatascience.com/about-joins-in-spark-3-0-1e0ea083ea86) for more information). Did the query become faster or slower?

In [ ]:
# your code goes here
df_operations \
    .join(df_aircraft.hint('shuffle_merge'), on='AirCraft', how='inner') \
    .groupby('AirCraftType') \
    .agg(F.avg('TimeOnTarget').alias('avg_tot')) \
    .sort(F.asc('avg_tot')) \
    .toPandas()

# Additional material

* An animated timeline of bombing operations is available [here](https://cdn.filepicker.io/api/file/el7zpgfhTOqapUblTvUh) and discussions about the dataset [here](https://data.world/datamil/vietnam-war-thor-data/discuss/vietnam-war-thor-data/gftdgyjz).